# Randomness and reproducibility

Random numbers and [stochastic processes](http://www2.econ.iastate.edu/tesfatsi/ace.htm#Stochasticity)
are essential to most agent-based models.
[Pseudo-random number generators](https://en.wikipedia.org/wiki/Pseudorandom_number_generator)
can be used to create numbers in a sequence that appears 
random but is actually a deterministic sequence based on an initial seed value.
In other words, the generator will produce the same pseudo-random sequence 
over multiple runs if it is given the same seed at the beginning.
Note that is possible that the generators will draw the same number repeatedly, 
as illustrated in this [comic strip](https://dilbert.com/strip/2001-10-25) from Scott Adams:

![Alt text](graphics/dilbert_rng.gif)

In [1]:
import agentpy as ap
import numpy as np
import random

## Random number generators

Agentpy models contain two internal pseudo-random number generators with different features:

- `Model.random` is an instance of `random.Random` (more info [here](https://realpython.com/python-random/))
- `Model.nprandom` is an instance of `numpy.random.Generator` (more info [here](https://numpy.org/devdocs/reference/random/index.html))

To illustrate, let us define a model that uses both generators to draw a random integer:

In [2]:
class RandomModel(ap.Model):
    
    def setup(self):
        self.x = self.random.randint(0, 99)
        self.y = self.nprandom.integers(99)
        self.report(['x', 'y'])
        self.stop() 

If we run this model multiple times, we will likely get a different series of numbers in each iteration:

In [3]:
exp = ap.Experiment(RandomModel, iterations=5)
results = exp.run()

Scheduled runs: 5
Completed: 5, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.013027


In [4]:
results.reporters

,x,y
iteration,,
0,92,11
1,1,14
2,73,13
3,51,13
4,46,79


## Defining custom seeds

If we want the results to be reproducible, 
we can define a parameter `seed` that 
will be used automatically at the beginning of a simulation
to initialize both generators.

In [5]:
parameters = {'seed': 42}
exp = ap.Experiment(RandomModel, parameters, iterations=5)
results = exp.run()

Scheduled runs: 5
Completed: 5, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.024486


By default, the experiment will use this seed to generate different random seeds for each iteration:

In [6]:
results.reporters

,x,y
iteration,,
0,26,68
1,70,9
2,58,66
3,48,77
4,94,65


Repeating this experiment will yield the same results:

In [7]:
exp2 = ap.Experiment(RandomModel, parameters, iterations=5)
results2 = exp2.run()

Scheduled runs: 5
Completed: 5, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.019584


In [8]:
results2.reporters

,x,y
iteration,,
0,26,68
1,70,9
2,58,66
3,48,77
4,94,65


Alternatively, we can set the argument `randomize=False` so that the experiment will use the same seed for each iteration:

In [9]:
exp3 = ap.Experiment(RandomModel, parameters, iterations=5, randomize=False)
results3 = exp3.run()

Scheduled runs: 5
Completed: 5, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.017925


Now, each iteration yields the same results:

In [10]:
results3.reporters

,x,y
iteration,,
0,35,39
1,35,39
2,35,39
3,35,39
4,35,39


## Sampling seeds

For a sample with multiple parameter combinations, we can treat the seed like any other parameter.
The following example will use the same seed for each parameter combination:

In [11]:
parameters = {'p': ap.Values(0, 1), 'seed': 0}
sample1 = ap.Sample(parameters, randomize=False)
list(sample1)

[{'p': 0, 'seed': 0}, {'p': 1, 'seed': 0}]

If we run an experiment with this sample,
the same iteration of each parameter combination will have the same seed (remember that the experiment will generate different seeds for each iteration by default):

In [12]:
exp = ap.Experiment(RandomModel, sample1, iterations=2)
results = exp.run()

Scheduled runs: 4
Completed: 4, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.025803


In [13]:
results.reporters

x   y
sample_id iteration        
0         0          68  31
          1          55  30
1         0          68  31
          1          55  30

Alternatively, we can use `Sample` with `randomize=True` (default)
to generate random seeds for each parameter combination in the sample.

In [14]:
sample3 = ap.Sample(parameters, randomize=True)
list(sample3)

[{'p': 0, 'seed': 302934307671667531413257853548643485645},
 {'p': 1, 'seed': 328530677494498397859470651507255972949}]

This will always generate the same set of random seeds:

In [15]:
sample3 = ap.Sample(parameters)
list(sample3)

[{'p': 0, 'seed': 302934307671667531413257853548643485645},
 {'p': 1, 'seed': 328530677494498397859470651507255972949}]

An experiment will now have different results for every parameter combination and iteration:

In [16]:
exp = ap.Experiment(RandomModel, sample3, iterations=2)
results = exp.run()

Scheduled runs: 4
Completed: 4, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.020825


In [17]:
results.reporters

x   y
sample_id iteration        
0         0          53  18
          1           3  60
1         0          83  62
          1          80  68

Repeating this experiment will yield the same results:

In [18]:
exp = ap.Experiment(RandomModel, sample3, iterations=2)
results = exp.run()

Scheduled runs: 4
Completed: 4, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.025660


In [19]:
results.reporters

x   y
sample_id iteration        
0         0          53  18
          1           3  60
1         0          83  62
          1          80  68

## Stochastic methods of AgentList

Let us now look at some stochastic operations that are often used in agent-based models. 
To start, we create a list of five agents:

In [20]:
model = ap.Model()
agents = ap.AgentList(model, 5)

In [21]:
agents

AgentList (5 objects)

If we look at the agent's ids, we see that they have been created in order:

In [22]:
agents.id

[1, 2, 3, 4, 5]

To shuffle this list, we can use `AgentList.shuffle`:

In [23]:
agents.shuffle().id

[5, 1, 2, 4, 3]

To create a random subset, we can use `AgentList.random`:

In [24]:
agents.random(3).id

[1, 3, 4]

And if we want it to be possible to select the same agent more than once:

In [25]:
agents.random(6, replace=True).id

[5, 4, 1, 2, 1, 3]

## Agent-specific generators

For more advanced applications, we can create separate generators for each object.
We can ensure that the seeds of each object follow a controlled pseudo-random sequence by using the models' main generator to generate the seeds.

In [26]:
class RandomAgent(ap.Agent):
    
    def setup(self):
        seed = self.model.random.getrandbits(128) # Seed from model
        self.random = random.Random(seed)  # Create agent generator
        self.x = self.random.random()  # Create a random number
        
class MultiRandomModel(ap.Model):
    
    def setup(self):
        self.agents = ap.AgentList(self, 2, RandomAgent)
        self.agents.record('x')
        self.stop()

In [27]:
parameters = {'seed': 42}
exp = ap.Experiment(
    MultiRandomModel, parameters, iterations=2, 
    record=True, randomize=False)
results = exp.run()

Scheduled runs: 2
Completed: 2, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.026438


In [28]:
results.variables.RandomAgent

x
iteration obj_id t          
0         1      0  0.414688
          2      0  0.591608
1         1      0  0.414688
          2      0  0.591608

Alternatively, we can also have each agent start from the same seed:

In [29]:
class RandomAgent2(ap.Agent):
    
    def setup(self):
        self.random = random.Random(self.p.agent_seed)  # Create agent generator
        self.x = self.random.random()  # Create a random number
        
class MultiRandomModel2(ap.Model):
    
    def setup(self):
        self.agents = ap.AgentList(self, 2, RandomAgent2)
        self.agents.record('x')
        self.stop()

In [30]:
parameters = {'agent_seed': 42}
exp = ap.Experiment(
    MultiRandomModel2, parameters, iterations=2, 
    record=True, randomize=False)
results = exp.run()

Scheduled runs: 2
Completed: 2, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.013392


In [31]:
results.variables.RandomAgent2

x
iteration obj_id t          
0         1      0  0.639427
          2      0  0.639427
1         1      0  0.639427
          2      0  0.639427